In [5]:
### Add IRI classification to RelevanC data

import pandas as pd
import s3fs
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': 'http://minio.stable.innovation.insee.eu'})

In [7]:
with open('product_sample.csv') as f:
    data_product = pd.read_csv(f)

data_product.dropna(inplace=True)    
data_product.code_ean.astype(int)

0               9426317
1         3041090050114
2         3371010174744
3         3470410134546
4         3533630060221
              ...      
565558    3830067420506
565559    3760077985507
565560    5060307410938
565564    3830067420070
565565    3266980197469
Name: code_ean, Length: 521050, dtype: int64

In [8]:
def lire_csv(file):
    with fs.open(file,'r',encoding='latin1') as f:
        data = pd.read_csv(f,delimiter=';',usecols=usecols)
    return(data)

list_fichiers=fs.ls('s3://groupe-1002/data/ref_art/20200118/')
usecols = ['DESCRIPTION FAMILLE','DESCRIPTION EAN','EAN']

In [11]:
data_fin = data_product
i = 0
for file in list_fichiers:
    i+=1
    if i % 10 == 0:
        print(i)
    data_iri = lire_csv(file)
    data_iri.EAN.astype(int)
    data_fin = pd.merge(data_fin, data_iri, left_on='code_ean', right_on='EAN', how='left')
    data_fin.loc[data_fin['DESCRIPTION FAMILLE'].notna(),'FAMILLE'] = data_fin.loc[data_fin['DESCRIPTION FAMILLE'].notna(),'DESCRIPTION FAMILLE']
    data_fin = data_fin[['code_ean','LIBELLE','FAMILLE']]

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770


In [15]:
data_fin.shape
data_fin.isna().sum()
data_fin.dropna(inplace=True)
data_fin.shape

(509153, 3)

In [30]:
replace_values = {'MONOPRIX' : '__ENSEIGNE___', 'MONOP' : '__ENSEIGNE__',
                      'MGOURMET' : '__ENSEIGNE__', 'CASINO' : '__ENSEIGNE__',
                      'FRANPRIX' : '__ENSEIGNE__'} 
data_fin.replace({'LIBELLE': replace_values},regex=True,inplace=True)

from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data_fin, train_size=100000)
data = train_data[['LIBELLE','FAMILLE']]

In [33]:

data.to_csv('out.csv',index=False)

In [32]:
data.head()

,LIBELLE,FAMILLE
421570,X3 BODY US ML ICONIQUE,LAYETTE BEBE
221779,6TR.LE TRANCHé FIN BOEUF+1TR G,JAMBON CUIT AU R FR
414986,RIGATONI 500G N 19,PATES ALIMENTAIRES
441312,BALLERINE BICOLORE,CHAUSSURE
225714,CHEM CRX FOR P 840 5A,CONFECTION ENFANT
